In [1]:
import sys

PWD = '../../..'
sys.path.append(PWD)

import pickle
import numpy as np
import pandas as pd

import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
import keras_tuner as kt

from sklearn.model_selection import train_test_split

from IPython.display import display, HTML

from src.network_utils import cross_validate_top_hyperparameters, count_params

In [2]:
DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + '/data/tuner'

PLANE = 2
CHANNEL = 11

PROJECT_NAME = f'mlp_{PLANE}_{CHANNEL}'

MAX_TRIALS = 50

TOP_N = 5
LOSS_WEIGHT = 1000

In [3]:
with open(DATASET_PATH, 'rb') as file:
    dataset = pickle.load(file)

all_X, all_y = dataset[(PLANE, CHANNEL)][0], dataset[(PLANE, CHANNEL)][1]
all_X.shape, all_y.shape

((28015, 24), (28015,))

In [4]:
X_base_train, _, y_base_train, _ = train_test_split(all_X, all_y, test_size=0.2,
                                                    random_state=42)  # Don't use the validation set

X_base_train.shape, y_base_train.shape

((22412, 24), (22412,))

# Model

In [5]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    hp_n_hidden_layers = hp.Int("n_hidden_layers", min_value=1, max_value=6, step=1, default=2)
    hp_units_mult = hp.Choice("units_mult", values=[1, 2, 4, 8, 16, 32], default=4)
    hp_lr = hp.Choice('learning_rate', values=[0.1, 0.01], default=0.01)
    hp_batch_normalization = hp.Boolean("batch_normalization")
    hp_dropout = hp.Choice("dropout", values=[0.0, 0.1, 0.2])

    model = keras.Sequential()
    model.add(layers.Input(24))

    n_units = 3 * (2 ** (hp_n_hidden_layers - 1)) * hp_units_mult
    for _ in range(hp_n_hidden_layers):
        if hp_batch_normalization:
            model.add(layers.BatchNormalization())
        model.add(layers.Dense(n_units, activation='relu'))
        n_units //= 2
        if hp_dropout > 0:
            model.add(layers.Dropout(hp_dropout))

    model.add(layers.Dense(1))

    model.compile(loss='mse', optimizer=optimizers.Adam(hp_lr), loss_weights=LOSS_WEIGHT)

    return model

In [6]:
model_builder(kt.HyperParameters()).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                600       
                                                                 
 dense_1 (Dense)             (None, 12)                300       
                                                                 
 dense_2 (Dense)             (None, 1)                 13        
                                                                 
Total params: 913
Trainable params: 913
Non-trainable params: 0
_________________________________________________________________


In [7]:
N_EPOCHS = 3000
BATCH_SIZE = 2048

model_callbacks = [
    callbacks.EarlyStopping(patience=50),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=10),
    callbacks.TensorBoard("/tmp/tb_logs/bayesian/" + PROJECT_NAME)
]

# Bayessian tuner

In [8]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', max_trials=MAX_TRIALS,
                                         directory=TRIALS_DIR + "/bayesian", project_name=PROJECT_NAME, overwrite=False)

bayesian_tuner.search(X_base_train, y_base_train, validation_split=0.2, epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

# Results

In [9]:
bayesian_tuner.results_summary(1)

Results summary
Results in ../../../data/tuner/bayesian\mlp_2_11
Showing 1 best trials
Trial summary
Hyperparameters:
n_hidden_layers: 5
units_mult: 8
learning_rate: 0.01
batch_normalization: False
dropout: 0.0
Score: 8.566548347473145


In [10]:
best_model = bayesian_tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 384)               9600      
                                                                 
 dense_1 (Dense)             (None, 192)               73920     
                                                                 
 dense_2 (Dense)             (None, 96)                18528     
                                                                 
 dense_3 (Dense)             (None, 48)                4656      
                                                                 
 dense_4 (Dense)             (None, 24)                1176      
                                                                 
 dense_5 (Dense)             (None, 1)                 25        
                                                                 
Total params: 107,905
Trainable params: 107,905
Non-trai

---

In [11]:
top_models = bayesian_tuner.get_best_models(num_models=TOP_N)
for i, model in enumerate(top_models):
    display(HTML(f"<h3>Model {i}</h3>"))
    print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 384)               9600      
                                                                 
 dense_1 (Dense)             (None, 192)               73920     
                                                                 
 dense_2 (Dense)             (None, 96)                18528     
                                                                 
 dense_3 (Dense)             (None, 48)                4656      
                                                                 
 dense_4 (Dense)             (None, 24)                1176      
                                                                 
 dense_5 (Dense)             (None, 1)                 25        
                                                                 
Total params: 107,905
Trainable params: 107,905
Non-trai

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 24)               96        
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 1536)              38400     
                                                                 
 dropout (Dropout)           (None, 1536)              0         
                                                                 
 batch_normalization_1 (Batc  (None, 1536)             6144      
 hNormalization)                                                 
                                                                 
 dense_1 (Dense)             (None, 768)               1180416   
                                                                 
 dropout_1 (Dropout)         (None, 768)               0

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 24)               96        
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 3072)              76800     
                                                                 
 dropout (Dropout)           (None, 3072)              0         
                                                                 
 batch_normalization_1 (Batc  (None, 3072)             12288     
 hNormalization)                                                 
                                                                 
 dense_1 (Dense)             (None, 1536)              4720128   
                                                                 
 dropout_1 (Dropout)         (None, 1536)              0

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 384)               9600      
                                                                 
 dense_1 (Dense)             (None, 192)               73920     
                                                                 
 dense_2 (Dense)             (None, 96)                18528     
                                                                 
 dense_3 (Dense)             (None, 48)                4656      
                                                                 
 dense_4 (Dense)             (None, 24)                1176      
                                                                 
 dense_5 (Dense)             (None, 1)                 25        
                                                                 
Total params: 107,905
Trainable params: 107,905
Non-trai

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 384)               9600      
                                                                 
 dense_1 (Dense)             (None, 192)               73920     
                                                                 
 dense_2 (Dense)             (None, 96)                18528     
                                                                 
 dense_3 (Dense)             (None, 48)                4656      
                                                                 
 dense_4 (Dense)             (None, 24)                1176      
                                                                 
 dense_5 (Dense)             (None, 1)                 25        
                                                                 
Total params: 107,905
Trainable params: 107,905
Non-trai

In [12]:
bayesian_tuner.results_summary(TOP_N)

Results summary
Results in ../../../data/tuner/bayesian\mlp_2_11
Showing 5 best trials
Trial summary
Hyperparameters:
n_hidden_layers: 5
units_mult: 8
learning_rate: 0.01
batch_normalization: False
dropout: 0.0
Score: 8.566548347473145
Trial summary
Hyperparameters:
n_hidden_layers: 5
units_mult: 32
learning_rate: 0.01
batch_normalization: True
dropout: 0.1
Score: 8.569682121276855
Trial summary
Hyperparameters:
n_hidden_layers: 6
units_mult: 32
learning_rate: 0.01
batch_normalization: True
dropout: 0.2
Score: 8.57243824005127
Trial summary
Hyperparameters:
n_hidden_layers: 5
units_mult: 8
learning_rate: 0.01
batch_normalization: False
dropout: 0.0
Score: 8.578202247619629
Trial summary
Hyperparameters:
n_hidden_layers: 5
units_mult: 8
learning_rate: 0.01
batch_normalization: False
dropout: 0.0
Score: 8.578230857849121


# Cross-validation for top 5 models

In [13]:
model_scores = cross_validate_top_hyperparameters(bayesian_tuner, X_base_train, y_base_train, model_builder,
                                                  n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, n_top=TOP_N)

{'n_hidden_layers': 5, 'units_mult': 8, 'learning_rate': 0.01, 'batch_normalization': False, 'dropout': 0.0}
Number of parameters: 107905
Got score: 8.83535385131836
Got score: 8.4826021194458
Got score: 8.116403579711914
Got score: 8.481605529785156
Got score: 8.889591217041016


{'n_hidden_layers': 5, 'units_mult': 32, 'learning_rate': 0.01, 'batch_normalization': True, 'dropout': 0.1}
Number of parameters: 1612465
Got score: 8.679004669189453
Got score: 8.33881664276123
Got score: 8.284358978271484
Got score: 8.549331665039062
Got score: 8.8145170211792


{'n_hidden_layers': 6, 'units_mult': 32, 'learning_rate': 0.01, 'batch_normalization': True, 'dropout': 0.2}
Number of parameters: 6377137
Got score: 8.807421684265137
Got score: 8.34884262084961
Got score: 8.27306842803955
Got score: 8.456929206848145
Got score: 8.83283805847168


{'n_hidden_layers': 5, 'units_mult': 8, 'learning_rate': 0.01, 'batch_normalization': False, 'dropout': 0.0}
Number of parameters: 107905
Got score: 8.781303405761719
Got score: 8.448953628540039
Got score: 8.125412940979004
Got score: 8.482863426208496
Got score: 8.819429397583008


{'n_hidden_layers': 5, 'units_mult': 8, 'learning_rate': 0.01, 'batch_normalization': False, 'dropout': 0.0}
Number of parameters: 107905
Got score: 8.785039901733398
Got score: 9.145454406738281
Got score: 8.107912063598633
Got score: 8.8272066116333
Got score: 8.867376327514648


In [14]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,8.56,0.28,107905
1,8.53,0.20,1612465
2,8.54,0.23,6377137
3,8.53,0.25,107905
4,8.75,0.34,107905
